# Playing around with Enformer and enformer_loader

## We can run VSCode in the farm!

In [2]:
import os

cwd = os.getcwd()
print(cwd)

/lustre/scratch126/gengen/teams/parts/sb79/enformer_loader_sgb


In [3]:
import socket

node_name = socket.gethostname()
print(node_name)

node-13-17


In [7]:
import multiprocess
cores = multiprocess.cpu_count()
print("Cores Available: " + str(cores))

Cores Available: 256


In [ ]:
# Test R code to check connection to a node
print("Hello, R is running successfully!")

# Check the R version
R.version.string

# Check the current working directory
getwd()
# Generate a simple plot
x <- rnorm(100)
y <- rnorm(100)
plot(x, y, main="Scatter plot of random points", xlab="X-axis", ylab="Y-axis")
quartz()  # Opens a new window for the plot (on Linux or macOS)
plot(1:10, 1:10)

R code is not that well suited to VSCode... but should be able to write scripts here anyway :)

## Let's inspect the enformer_loader output

In [16]:
# Import necessary packages
import numpy as np
import sys
import os
import pandas as pd
from Bio import SeqIO
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
import math
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import re
import csv

/software/hgi/installs/conda-audited/miniforge/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# let's import the .bed file from enformer_loader
path = 'tests/data/'
bed_df = pd.read_csv(path + 'test_dataset.bed', sep='\t', header=None)
bed_df

,0,1,2,3
0,chr1,880,920,"[1.0, 1.0]"
1,chr1,493,533,"[1.0, 1.0]"
2,chr2,390,430,"[2.200000047683716, 3.200000047683716]"
3,chr1,92,132,"[1.0, 1.0]"
4,chr1,458,498,"[1.0, 1.0]"
...,...,...,...,...
95,chr3,563,603,"[2.0, 2.0]"
96,chr1,319,359,"[1.0, 1.0]"
97,chr2,477,517,"[3.200000047683716, 3.200000047683716]"
98,chr2,21,61,"[0.800000011920929, 0.800000011920929]"


In [20]:
# see how we could generate a train/val/test dataset for enformer


NameError: name 'scripts' is not defined